# DESCENSO AL FONDO DE UN CRÁTER EN MARTE

Yuu Ricardo Akachi Tanaka | A01351969

Pablo Monzón Terrazas | A01562619

Donnet Emmanuel Hernández Franco | A01352049

## Búsqueda codiciosa: Hill Climbing

In [ ]:
#------------------------------------------------------------------------------------------------------------------
#   Hill Climbing solver para robot explorador en Marte para el descenso al fondo de un cráter
#------------------------------------------------------------------------------------------------------------------

#------------------------------------------------------------------------------------------------------------------
#   Imports
#------------------------------------------------------------------------------------------------------------------
import time
import random
import math
import numpy as np

#------------------------------------------------------------------------------------------------------------------
#   Función para sacar altura
#------------------------------------------------------------------------------------------------------------------
def Altura(x,y):
  r = nr-round(y/10.045) # row escalado
  c = round(x/10.045) # column escalado
  return mapa[r][c]

#------------------------------------------------------------------------------------------------------------------
#   Variables
#------------------------------------------------------------------------------------------------------------------
mapa = np.load('crater_map.npy') # se carga el mapa del cráter
nr, nc = mapa.shape # se saca el tamaño del mapa

# Establecemos límites del mapa
x_todos = np.round(10.045*np.arange(mapa.shape[1]))
y_todos = np.round(10.045*np.arange(mapa.shape[0]))
x_min = x_todos[0]
x_max = x_todos[-1]
y_min = y_todos[0]
y_max = y_todos[-1]

#------------------------------------------------------------------------------------------------------------------
#   Class definitions
#------------------------------------------------------------------------------------------------------------------

class Crater(object):
    """
        Clase que representa el mapa de la superficie de marte con un cráter. En donde
        cada coordenada contiene su altura. El mapa de altura está en el archivo
        crater_map.npy que tiene los datos por ROW y COLUMN
    """

    def __init__(self, x, y, hmax):
        """
            El constructor inicializa el mapa con el rover en una posición
        """

        z = Altura(x, y)# altura

        self.pos_i = (x, y, z) # crea vector posición

        self.escala = 10.045
        self.hmax = hmax # altura máxima que se puede mover


    def show(self):
        """ Este método imprime la posición actual de la matriz. """
        print('Estás en la posición (',self.pos_i[0], ',', self.pos_i[1], ',', self.pos_i[2], ')')

    def cost(self):
        """ Esta función calcula el costo de la solución (la altura en la que se encuentra)"""

        return self.pos_i[2]

    def best_neighbor(self):
        """ Esta función regresa la mejor posición a la que se movió
            Se puede mover a uno de los 8 vecinos que rodea al robot
        """
        neighbor=[]

        # Copia el estado actual
        new_pos = Crater(self.pos_i[0], self.pos_i[1], self.hmax)

        # Checa todos sus vecinos
        ## DERECHA
        if self.pos_i[0] < x_max:
          if abs(Altura(round(self.pos_i[0] + self.escala), self.pos_i[1]) - self.pos_i[2]) <= self.hmax:
            z = Altura(round(self.pos_i[0] + self.escala), self.pos_i[1])
            neighbor.append((round(self.pos_i[0] + self.escala), self.pos_i[1], z))


        ## IZQUIERDA
        if self.pos_i[0] > x_min:
          if abs(Altura(round(self.pos_i[0] - self.escala), self.pos_i[1]) - self.pos_i[2]) <= self.hmax:
            z = Altura(round(self.pos_i[0] - self.escala), self.pos_i[1])
            neighbor.append((round(self.pos_i[0] - self.escala), self.pos_i[1], z))


        ## ARRIBA
        if self.pos_i[1] < y_max:
          if abs(Altura(self.pos_i[0], round(self.pos_i[1] + self.escala)) - self.pos_i[2]) <= self.hmax:
            z = Altura(self.pos_i[0], round(self.pos_i[1] + self.escala))
            neighbor.append((self.pos_i[0], round(self.pos_i[1] + self.escala), z))


        ## ABAJO
        if self.pos_i[1] > y_min:
          if abs(Altura(self.pos_i[0], round(self.pos_i[1] - self.escala)) - self.pos_i[2]) <= self.hmax:
            z = Altura(self.pos_i[0], round(self.pos_i[1] - self.escala))
            neighbor.append((self.pos_i[0], round(self.pos_i[1] - self.escala), z))


        ## ARRIBA DERECHA
        if (self.pos_i[0] < x_max) and (self.pos_i[1] < y_max):
          if abs(Altura(round(self.pos_i[0] + self.escala), round(self.pos_i[1] + self.escala)) - self.pos_i[2]) <= self.hmax:
            z = Altura(round(self.pos_i[0] + self.escala), round(self.pos_i[1] + self.escala))
            neighbor.append((round(self.pos_i[0] + self.escala), round(self.pos_i[1] + self.escala), z))


        ## ARRIBA IZQUIERDA
        if (self.pos_i[0] > x_min) and (self.pos_i[1] < y_max):
          if abs(Altura(round(self.pos_i[0] - self.escala), round(self.pos_i[1] + self.escala)) - self.pos_i[2]) <= self.hmax:
            z = Altura(round(self.pos_i[0] - self.escala), round(self.pos_i[1] + self.escala))
            neighbor.append((round(self.pos_i[0] - self.escala), round(self.pos_i[1] + self.escala), z))


        ## ABAJO DERECHA
        if (self.pos_i[0] < x_max) and (self.pos_i[1] > y_min):
          if abs(Altura(round(self.pos_i[0] + self.escala), round(self.pos_i[1] - self.escala)) - self.pos_i[2]) <= self.hmax:
            z = Altura(round(self.pos_i[0] + self.escala), round(self.pos_i[1] - self.escala))
            neighbor.append((round(self.pos_i[0] + self.escala), round(self.pos_i[1] - self.escala), z))


        ## ABAJO IZQUIERDA
        if (self.pos_i[0] > x_min) and (self.pos_i[1] > y_min):
          if abs(Altura(round(self.pos_i[0] - self.escala), round(self.pos_i[1] - self.escala)) - self.pos_i[2]) <= self.hmax:
            z = Altura(round(self.pos_i[0] - self.escala), round(self.pos_i[1] - self.escala))
            neighbor.append((round(self.pos_i[0] - self.escala), round(self.pos_i[1] - self.escala), z))

        # Checamos cuál es el mejor
        h_min = self.pos_i[2]
        if any(i[2] <= h_min for i in neighbor):
          for i in neighbor:
              if i[2] <= h_min:
                h_min = i[2]
                new_pos.pos_i = i

        return new_pos

#------------------------------------------------------------------------------------------------------------------
#   Program
#------------------------------------------------------------------------------------------------------------------
#random.seed(time.time()*1000)

x_i = 3570
y_i = 6980
altura_max = 2

map = Crater(x_i, y_i, altura_max)      # Initialize board
map.show()

cost = map.cost()         # Initial cost
print("Altura inicial: ", cost)

step = 0                    # Step count

while True:
    step += 1

    # Get best neighbor
    neighbor = map.best_neighbor()
    new_cost = neighbor.cost()

    # Test neighbor
    if new_cost < cost:
      map = neighbor
      cost = new_cost
    else:
      break


    print("Iteration: ", step, "    Altura: ", cost)

print("\n--------Solution-----------")
map.show()


#------------------------------------------------------------------------------------------------------------------
#   End of file
#------------------------------------------------------------------------------------------------------------------

Estás en la posición ( 3570 , 6980 , 137.6156396484377 )
Altura inicial:  137.6156396484377
Iteration:  1     Altura:  136.10144042968773
Iteration:  2     Altura:  134.62615966796898
Iteration:  3     Altura:  133.1761230468752
Iteration:  4     Altura:  131.6671240234377
Iteration:  5     Altura:  129.86455566406272
Iteration:  6     Altura:  128.92862304687523
Iteration:  7     Altura:  127.63760742187522
Iteration:  8     Altura:  127.34269531250021
Iteration:  9     Altura:  126.86796630859396
Iteration:  10     Altura:  126.20543701171897
Iteration:  11     Altura:  125.47180419921897
Iteration:  12     Altura:  125.00404052734397
Iteration:  13     Altura:  124.58188964843772
Iteration:  14     Altura:  124.23390625000022
Iteration:  15     Altura:  124.07245117187522
Iteration:  16     Altura:  124.02054687500022
Iteration:  17     Altura:  123.94737792968772
Iteration:  18     Altura:  123.78072509765647
Iteration:  19     Altura:  123.53465576171897
Iteration:  20     Altura:

## Recocido simulado

In [ ]:
#------------------------------------------------------------------------------------------------------------------
#   Recocido Simulado para robot explorador en Marte para el descenso al fondo de un cráter
#------------------------------------------------------------------------------------------------------------------

#------------------------------------------------------------------------------------------------------------------
#   Imports
#------------------------------------------------------------------------------------------------------------------
import time
import random
import math
import numpy as np

#------------------------------------------------------------------------------------------------------------------
#   Función para sacar altura
#------------------------------------------------------------------------------------------------------------------
def Altura(x,y):
  r = nr-round(y/10.045) # row escalado
  c = round(x/10.045) # column escalado
  return mapa[r][c]

#------------------------------------------------------------------------------------------------------------------
#   Variables
#------------------------------------------------------------------------------------------------------------------
mapa = np.load('crater_map.npy') # se carga el mapa del cráter
nr, nc = mapa.shape # se saca el tamaño del mapa

# Establecemos límites del mapa
x_todos = np.round(10.045*np.arange(mapa.shape[1]))
y_todos = np.round(10.045*np.arange(mapa.shape[0]))
x_min = x_todos[0]
x_max = x_todos[-1]
y_min = y_todos[0]
y_max = y_todos[-1]

#------------------------------------------------------------------------------------------------------------------
#   Class definitions
#------------------------------------------------------------------------------------------------------------------

class Crater(object):
    """
        Clase que representa el mapa de la superficie de marte con un cráter. En donde
        cada coordenada contiene su altura. El mapa de altura está en el archivo
        crater_map.npy que tiene los datos por ROW y COLUMN
    """

    def __init__(self, x, y, hmax):
        """
            El constructor inicializa el mapa con el rover en una posición
        """

        z = Altura(x, y)# altura

        self.pos_i = (x, y, z) # crea vector posición

        self.escala = 10.045
        self.hmax = hmax # altura máxima que se puede mover


    def show(self):
        """ Este método imprime la posición actual de la matriz. """
        print('Estás en la posición (',self.pos_i[0], ',', self.pos_i[1], ',', self.pos_i[2], ')')

    def cost(self):
        """ Esta función calcula el costo de la solución (la altura en la que se encuentra)"""

        return self.pos_i[2]

    def neighbor(self):
        """ Esta función regresa la mejor posición a la que se movió
            Se puede mover a uno de los 8 vecinos que rodea al robot
        """

        # Copia el estado actual
        new = Crater(self.pos_i[0], self.pos_i[1], self.hmax)
        #new.pos_i = self.pos_i.copy()

        # Selecciona uno de los parametros de forma aleatoria
        i = random.randint(0,7)

        # Se mueve a la posición random
        ## DERECHA
        if i == 0 and self.pos_i[0] < x_max:
          if abs(Altura(round(self.pos_i[0] + self.escala), self.pos_i[1]) - self.pos_i[2]) <= self.hmax:
            z = Altura(round(self.pos_i[0] + self.escala), self.pos_i[1])
            new.pos_i = (round(self.pos_i[0] + self.escala), self.pos_i[1], z)

        ## IZQUIERDA
        if i == 1 and self.pos_i[0] > x_min:
          if abs(Altura(round(self.pos_i[0] - self.escala), self.pos_i[1]) - self.pos_i[2]) <= self.hmax:
            z = Altura(round(self.pos_i[0] - self.escala), self.pos_i[1])
            new.pos_i = (round(self.pos_i[0] - self.escala), self.pos_i[1], z)

        ## ARRIBA
        if i == 2 and self.pos_i[1] < y_max:
          if abs(Altura(self.pos_i[0], round(self.pos_i[1] + self.escala)) - self.pos_i[2]) <= self.hmax:
            z = Altura(self.pos_i[0], round(self.pos_i[1] + self.escala))
            new.pos_i = (self.pos_i[0], round(self.pos_i[1] + self.escala), z)

        ## ABAJO
        if i == 3 and self.pos_i[1] > y_min:
          if abs(Altura(self.pos_i[0], round(self.pos_i[1] - self.escala)) - self.pos_i[2]) <= self.hmax:
            z = Altura(self.pos_i[0], round(self.pos_i[1] - self.escala))
            new.pos_i = (self.pos_i[0], round(self.pos_i[1] - self.escala), z)

        ## ARRIBA DERECHA
        if i == 4 and (self.pos_i[0] < x_max) and (self.pos_i[1] < y_max):
          if abs(Altura(round(self.pos_i[0] + self.escala), round(self.pos_i[1] + self.escala)) - self.pos_i[2]) <= self.hmax:
            z = Altura(round(self.pos_i[0] + self.escala), round(self.pos_i[1] + self.escala))
            new.pos_i = (round(self.pos_i[0] + self.escala), round(self.pos_i[1] + self.escala), z)

        ## ARRIBA IZQUIERDA
        if i == 5 and (self.pos_i[0] > x_min) and (self.pos_i[1] < y_max):
          if abs(Altura(round(self.pos_i[0] - self.escala), round(self.pos_i[1] + self.escala)) - self.pos_i[2]) <= self.hmax:
            z = Altura(round(self.pos_i[0] - self.escala), round(self.pos_i[1] + self.escala))
            new.pos_i = (round(self.pos_i[0] - self.escala), round(self.pos_i[1] + self.escala), z)

        ## ABAJO DERECHA
        if i == 6 and (self.pos_i[0] < x_max) and (self.pos_i[1] > y_min):
          if abs(Altura(round(self.pos_i[0] + self.escala), round(self.pos_i[1] - self.escala)) - self.pos_i[2]) <= self.hmax:
            z = Altura(round(self.pos_i[0] + self.escala), round(self.pos_i[1] - self.escala))
            new.pos_i = (round(self.pos_i[0] + self.escala), round(self.pos_i[1] - self.escala), z)

        ## ABAJO IZQUIERDA
        if i == 7 and (self.pos_i[0] > x_min) and (self.pos_i[1] > y_min):
          if abs(Altura(round(self.pos_i[0] - self.escala), round(self.pos_i[1] - self.escala)) - self.pos_i[2]) <= self.hmax:
            z = Altura(round(self.pos_i[0] - self.escala), round(self.pos_i[1] - self.escala))
            new.pos_i = (round(self.pos_i[0] - self.escala), round(self.pos_i[1] - self.escala), z)

        return new

#------------------------------------------------------------------------------------------------------------------
#   Program
#------------------------------------------------------------------------------------------------------------------
#random.seed(time.time()*1000)

x_i = 3570 # x inicial
y_i = 6980 # y inicial
altura_max = 2 # altura max que puede recorrer

crater = Crater(x_i, y_i, altura_max)      # Initialize board
crater.show()

cost = crater.cost()         # Initial cost
print("Altura Inicial: ", cost)

step = 0                    # Step count

alpha = 0.9995              # Coefficient of the exponential temperature schedule
t0 = 1                      # Initial temperature
t = t0

while t > 0.005 and cost > 0:

    # Calculate temperature
    t = t0 * math.pow(alpha, step)
    #t -=0.001
    step += 1

    # Get random neighbor
    neighbor = crater.neighbor()
    new_cost = neighbor.cost()

    # Test neighbor
    if new_cost < cost:
        crater = neighbor
        cost = new_cost
    else:
        # Calculate probability of accepting the neighbor
        p = math.exp(-(new_cost - cost)/t)
        if p >= random.random():
            crater = neighbor
            cost = new_cost

    print("Iteration: ", step, "    Altura: ", cost, "    Temperature: ", t)

print("\n--------Solution-----------")
crater.show()
#print(crater.parametros)

#------------------------------------------------------------------------------------------------------------------
#   End of file
#------------------------------------------------------------------------------------------------------------------

Se han truncado las últimas 5000 líneas del flujo de salida.
Iteration:  5599     Altura:  123.32181640625022     Temperature:  0.06082830939430224
Iteration:  5600     Altura:  123.35913818359397     Temperature:  0.0607978952396051
Iteration:  5601     Altura:  123.35913818359397     Temperature:  0.060767496291985294
Iteration:  5602     Altura:  123.35913818359397     Temperature:  0.06073711254383931
Iteration:  5603     Altura:  123.32181640625022     Temperature:  0.06070674398756739
Iteration:  5604     Altura:  123.32181640625022     Temperature:  0.06067639061557361
Iteration:  5605     Altura:  123.32181640625022     Temperature:  0.06064605242026583
Iteration:  5606     Altura:  123.32181640625022     Temperature:  0.0606157293940557
Iteration:  5607     Altura:  123.32181640625022     Temperature:  0.060585421529358675
Iteration:  5608     Altura:  123.32181640625022     Temperature:  0.060555128818594
Iteration:  5609     Altura:  123.41948730468772     Temperature:  0.06